In [ ]:
#works with cv virtual environment

In [1]:
import numpy as np
# import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

import sys
import glob
import cv2

In [2]:
folder_path = '/scratch/hpatches/hpatches-benchmark/data/hpatches-release'
patch_size = 65

In [3]:
# all types of patches 
tps = ['ref','e1','e2','e3','e4','e5','h1','h2','h3','h4','h5',\
       't1','t2','t3','t4','t5']

class hpatches_sequence:
    """Class for loading an HPatches sequence from a sequence folder"""
    itr = tps
    def __init__(self,base):
        name = base.split('/')
        self.name = name[-1]
        self.base = base
        for t in self.itr:
            im_path = os.path.join(base, t+'.png')
            im = cv2.imread(im_path,0)
            self.N = im.shape[0]/patch_size
            setattr(self, t, np.split(im, self.N))

In [4]:
seqs = glob.glob(folder_path+'/*')
seqs = [os.path.abspath(p) for p in seqs]

In [5]:
len(seqs)

116

In [6]:
descr_name = 'chen'
base_dir = '/home/niaki/Projects/patch-desc-ae/other_descriptors'
model_version = 'encoderChenEtAl_400it_20210702.npy'

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
input_size = patch_size * patch_size
hidden_size = 128

In [9]:
theta = np.load(base_dir + '/' + model_version)
W1 = theta[0:hidden_size * input_size].reshape(hidden_size, input_size)
# W2 = theta[hidden_size * input_size:2 * hidden_size * input_size].reshape(input_size, hidden_size)
b1 = theta[2 * hidden_size * input_size:2 * hidden_size * input_size + hidden_size]
# b2 = theta[2 * hidden_size * input_size + hidden_size:]

In [10]:
# patch = np.zeros((65,65))

In [11]:
def compute_chen(patch):
    data = np.expand_dims(patch.reshape(patch_size * patch_size), axis=1)
    z2 = W1.dot(data) + np.tile(b1, (1, 1)).transpose()
    patch_encoding = sigmoid(z2)
    return patch_encoding

In [ ]:
# for seq_path in seqs:
#     seq = hpatches_sequence(seq_path)
#     path = os.path.join(descr_name,seq.name)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     descr = np.zeros((seq.N,2)) # trivial (mi,sigma) descriptor
#     for tp in tps:
#         print(seq.name+'/'+tp)
#         for i,patch in enumerate(getattr(seq, tp)):
#             mi = np.mean(patch) # trivial (mi,sigma) descriptor
#             sigma = np.std(patch) # trivial (mi,sigma) descriptor
#             descr[i] = np.array([mi,sigma])
#         np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

In [15]:
save_dir = '/scratch/cloned_repositories/hpatches-benchmark/data/descriptors/'

In [16]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join(save_dir, descr_name, seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = 128
    descr = np.zeros((int(seq.N),descr_size_per_patch))
    print(seq.name)
    for tp in tps:
#         print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
            patch = patch / 255.0
#             print(patch.shape)
            patch_encoded = compute_chen(patch)
#             print(patch_encoded.shape)
            patch_crop_encoded_flat = patch_encoded.flatten()
#             print(patch_crop_encoded_flat.shape)
#             print()
            descr[i] = patch_crop_encoded_flat
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',')

i_nijmegen


KeyboardInterrupt: 

In [23]:
seq.N

1999.0

In [25]:
path

'hpatches_extracted_descrs/chen/i_santuario'